# 기계학습 팀 소믈리에 와인 등급 예측
## kNN 0521
### KNeighborsRegressor 시도

#### 분류기 말고 회귀도 있다는 것을 찾아서 시도해본 것

In [1]:
import numpy as np
import pandas as pd

In [2]:
# csv를 가져와 넘파이배열 형태의 데이터셋으로 가공하는 과정
import csv
f1=open('winequality-red.csv','r',encoding='UTF-8')
f2=open('winequality-white.csv','r',encoding='UTF-8')
red_wine_data = csv.reader(f1)
white_wine_data = csv.reader(f2)
raw1=[]# 가공 전 레드와인 데이터를 넣어둘 리스트
raw2=[]# 가공 전 화이트와인 데이터를 넣어둘 리스트

total_data=[]# 가공된 모든 와인 데이터를 넣어둘 리스트

features=[]# 특성 이름을 모아둘 리스트
feat=[]# csv에 "특성이름" 의 형태로 들어가 있어서 ""을 제거하기 전 특성 이름 리스트
target=[]# 정답인 등급을 모아둘 리스트

for r in red_wine_data:
    r=r[0].split(';')
    r.insert(0,'1')#red가 1
    raw1.append(r)

for w in white_wine_data:
    w=w[0].split(';')
    w.insert(0,'0')#white가 0
    raw2.append(w)

raw1[0].pop(0)# 와인의 색 정보로 위에서 넣어둔 1을 빼냄
feat=raw1[0]
for f in feat:
    s=f[1:len(f)-1]# ""을 제거하는 부분
    features.append(s)
features.insert(0,'color')

# csv의 첫번째 행이 특성 정보이므로 그것을 없애는 작업
raw1.pop(0)
raw2.pop(0)

for i1 in raw1:
    total_data.append(i1)

for i2 in raw2:
    total_data.append(i2)

for t in total_data:
    target.append(t.pop())


total_data=np.array(total_data)
total_data = total_data.astype(np.float64)# 실행했을때 밑의 경고문이 떠서 추가
"""
FutureWarning:
Beginning in version 0.22,
arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'.
It is recommended that you convert the array to a float dtype before using it in scikit-learn
"""

import copy

target_5=copy.deepcopy(target) # 1~4 / 5~10
target_6=copy.deepcopy(target) # 1~5 / 6~10
target_7=copy.deepcopy(target) # 1~6 / 7~10

for i in range(0,len(target_5)):
    if int(target_5[i])<5:
        target_5[i]='0'
    else:
       target_5[i]='1'
        
for i in range(0,len(target_6)):
    if int(target_6[i])<5:
        target_6[i]='0'
    else:
       target_6[i]='1'
        
for i in range(0,len(target_7)):
    if int(target_7[i])<5:
        target_7[i]='0'
    else:
       target_7[i]='1'
        
target_5=np.array(target_5)
target_5 = target_5.astype(np.float64)
target_6=np.array(target_6)
target_6 = target_6.astype(np.float64)
target_7=np.array(target_7)
target_7 = target_7.astype(np.float64)

f1.close()
f2.close()

#위에서 copy가 일어나므로 밑에서 처리
target=np.array(target)
target = target.astype(np.float64)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

In [14]:
# k의 차이에 따른 결과를 보기 위해, 다른 데이터셋에 따른 결과를 보기 위해 편리하게 사용할 목적으로 정의한 함수
def knr_by_n(n,x,y,test,pred):
    #매개변수는 순서대로 k, 훈련데이터 특성배열, 훈련데이터 타겟배열, 테스트셋 특성배열, 테스트셋 타겟배열
    reg=KNeighborsRegressor(n_neighbors=n, weights='distance')#k=n인 knn
    reg.fit(x, y)
    print("n = ", n)
    print("테스트 세트 예측:\n{}".format(reg.predict(test)))
    print("테스트 세트 R^2: {:.2f}".format(reg.score(test, pred)))
    return reg.score(test, pred)

### 다중분류

In [5]:
# 훈련 데이터와 테스트 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(total_data, target, random_state=0)
# 훈련 데이터 크기 확인
print(x_train.shape)
print(y_train.shape)
# 테스트 데이터 크기 확인
print(x_test.shape)
print(y_test.shape)

(4872, 12)
(4872,)
(1625, 12)
(1625,)


In [6]:
for i in range(1,1625):
    #최대 k = 1625
    knr_by_n(i,x_train,y_train,x_test,y_test)

n =  1
테스트 세트 예측:
[6. 6. 6. ... 5. 6. 5.]
테스트 세트 R^2: -0.02
n =  2
테스트 세트 예측:
[6. 6. 6. ... 5. 6. 5.]
테스트 세트 R^2: 0.15
n =  3
테스트 세트 예측:
[5.6691215 6.        6.        ... 5.3096962 6.        5.       ]
테스트 세트 R^2: 0.22
n =  4
테스트 세트 예측:
[5.74709314 6.         5.78606775 ... 5.47270936 6.         5.        ]
테스트 세트 R^2: 0.26
n =  5
테스트 세트 예측:
[5.60575291 6.         5.64753841 ... 5.56369726 6.         5.        ]
테스트 세트 R^2: 0.28
n =  6
테스트 세트 예측:
[5.51238125 6.         5.55051986 ... 5.627905   6.         5.        ]
테스트 세트 R^2: 0.30
n =  7
테스트 세트 예측:
[5.5768553  6.         5.47878506 ... 5.67417307 6.         5.        ]
테스트 세트 R^2: 0.31
n =  8
테스트 세트 예측:
[5.50998399 6.         5.42992632 ... 5.70959963 6.         5.        ]
테스트 세트 R^2: 0.32
n =  9
테스트 세트 예측:
[5.56063833 6.         5.39011602 ... 5.73755729 6.         5.        ]
테스트 세트 R^2: 0.33
n =  10
테스트 세트 예측:
[5.50878383 6.         5.3571723  ... 5.75937126 6.         5.        ]
테스트 세트 R^2: 0.33
n =  11
테스트 세트 예측:
[5.54949263

n =  78
테스트 세트 예측:
[5.69725077 6.         5.59919411 ... 5.79443529 6.         5.        ]
테스트 세트 R^2: 0.34
n =  79
테스트 세트 예측:
[5.70974937 6.         5.59378072 ... 5.80455666 6.         5.        ]
테스트 세트 R^2: 0.34
n =  80
테스트 세트 예측:
[5.72201043 6.         5.58849593 ... 5.79791694 6.         5.        ]
테스트 세트 R^2: 0.34
n =  81
테스트 세트 예측:
[5.7246193  6.         5.58335087 ... 5.78320817 6.         5.        ]
테스트 세트 R^2: 0.34
n =  82
테스트 세트 예측:
[5.73634645 6.         5.578311   ... 5.7769018  6.         5.        ]
테스트 세트 R^2: 0.34
n =  83
테스트 세트 예측:
[5.73873052 6.         5.5734199  ... 5.77868022 6.         5.        ]
테스트 세트 R^2: 0.34
n =  84
테스트 세트 예측:
[5.741061   6.         5.56861266 ... 5.78042944 6.         5.        ]
테스트 세트 R^2: 0.34
n =  85
테스트 세트 예측:
[5.74334952 6.         5.56390852 ... 5.78996818 6.         5.        ]
테스트 세트 R^2: 0.34
n =  86
테스트 세트 예측:
[5.73687345 6.         5.55929815 ... 5.79159115 6.         5.        ]
테스트 세트 R^2: 0.34
n =  87
테스트 세트 예측:
[5.730597

[5.78980277 6.         5.49915028 ... 5.74547216 6.         5.        ]
테스트 세트 R^2: 0.33
n =  155
테스트 세트 예측:
[5.7862546  6.         5.49704884 ... 5.74218249 6.         5.        ]
테스트 세트 R^2: 0.33
n =  156
테스트 세트 예측:
[5.78273816 6.         5.49497241 ... 5.73892728 6.         5.        ]
테스트 세트 R^2: 0.33
n =  157
테스트 세트 예측:
[5.78814713 6.         5.49291326 ... 5.7357005  6.         5.        ]
테스트 세트 R^2: 0.33
n =  158
테스트 세트 예측:
[5.78466449 6.         5.49087352 ... 5.73250492 6.         5.        ]
테스트 세트 R^2: 0.33
n =  159
테스트 세트 예측:
[5.78121929 6.         5.48885194 ... 5.72934127 6.         5.        ]
테스트 세트 R^2: 0.33
n =  160
테스트 세트 예측:
[5.78217337 6.         5.48684929 ... 5.73050499 6.         5.        ]
테스트 세트 R^2: 0.33
n =  161
테스트 세트 예측:
[5.78311779 6.         5.48893918 ... 5.72741896 6.         5.        ]
테스트 세트 R^2: 0.33
n =  162
테스트 세트 예측:
[5.77973777 6.         5.4910067  ... 5.73277073 6.         5.        ]
테스트 세트 R^2: 0.33
n =  163
테스트 세트 예측:
[5.77640435 6.     

테스트 세트 R^2: 0.32
n =  231
테스트 세트 예측:
[5.74842734 6.         5.47248662 ... 5.72017796 6.         5.        ]
테스트 세트 R^2: 0.32
n =  232
테스트 세트 예측:
[5.74915284 6.         5.47119416 ... 5.72098481 6.         5.        ]
테스트 세트 R^2: 0.32
n =  233
테스트 세트 예측:
[5.74987252 6.         5.46992604 ... 5.72465812 6.         5.        ]
테스트 세트 R^2: 0.32
n =  234
테스트 세트 예측:
[5.74772789 6.         5.47134035 ... 5.72544641 6.         5.        ]
테스트 세트 R^2: 0.32
n =  235
테스트 세트 예측:
[5.75129485 6.         5.47274124 ... 5.72337551 6.         5.        ]
테스트 세트 R^2: 0.32
n =  236
테스트 세트 예측:
[5.75484155 6.         5.47149346 ... 5.72984288 6.         5.        ]
테스트 세트 R^2: 0.32
n =  237
테스트 세트 예측:
[5.75836591 6.         5.47025224 ... 5.7362736  6.         5.        ]
테스트 세트 R^2: 0.32
n =  238
테스트 세트 예측:
[5.7590458  6.         5.47688394 ... 5.73419605 6.         5.        ]
테스트 세트 R^2: 0.32
n =  239
테스트 세트 예측:
[5.75972057 6.         5.47823691 ... 5.73494131 6.         5.        ]
테스트 세트 R^2: 0.32
n 

테스트 세트 R^2: 0.32
n =  307
테스트 세트 예측:
[5.80759831 6.         5.49780357 ... 5.70922951 6.         5.        ]
테스트 세트 R^2: 0.32
n =  308
테스트 세트 예측:
[5.80592724 6.         5.49878819 ... 5.71196471 6.         5.        ]
테스트 세트 R^2: 0.32
n =  309
테스트 세트 예측:
[5.80839189 6.         5.49976784 ... 5.71468834 6.         5.        ]
테스트 세트 R^2: 0.32
n =  310
테스트 세트 예측:
[5.80878653 6.         5.50074368 ... 5.71107377 6.         5.        ]
테스트 세트 R^2: 0.32
n =  311
테스트 세트 예측:
[5.80712509 6.         5.50171523 ... 5.71168032 6.         5.        ]
테스트 세트 R^2: 0.32
n =  312
테스트 세트 예측:
[5.80751978 6.         5.50268291 ... 5.71228346 6.         5.        ]
테스트 세트 R^2: 0.32
n =  313
테스트 세트 예측:
[5.8058716  6.         5.5055847  ... 5.71288248 6.         5.        ]
테스트 세트 R^2: 0.32
n =  314
테스트 세트 예측:
[5.80830349 6.         5.50460859 ... 5.71347901 6.         5.        ]
테스트 세트 R^2: 0.32
n =  315
테스트 세트 예측:
[5.80869307 6.         5.50364025 ... 5.71407307 6.         5.        ]
테스트 세트 R^2: 0.32
n 

...

테스트 세트 R^2: 0.29
n =  1287
테스트 세트 예측:
[5.91474088 6.         5.62801932 ... 5.77224827 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1288
테스트 세트 예측:
[5.91477624 6.         5.62815299 ... 5.77189723 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1289
테스트 세트 예측:
[5.91481139 6.         5.62792749 ... 5.77154699 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1290
테스트 세트 예측:
[5.91484652 6.         5.62806098 ... 5.77165059 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1291
테스트 세트 예측:
[5.91405754 6.         5.62855297 ... 5.77220723 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1292
테스트 세트 예측:
[5.91409289 6.         5.62904461 ... 5.7718575  6.         5.        ]
테스트 세트 R^2: 0.29
n =  1293
테스트 세트 예측:
[5.91453896 6.         5.6295359  ... 5.77196068 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1294
테스트 세트 예측:
[5.91498467 6.         5.63002654 ... 5.77251576 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1295
테스트 세트 예측:
[5.91501952 6.         5.63051609 ... 5.77307025 6.         5.        ]
테스트 세트 R^2

테스트 세트 R^2: 0.29
n =  1362
테스트 세트 예측:
[5.92037151 6.         5.63248664 ... 5.77836044 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1363
테스트 세트 예측:
[5.92079037 6.         5.63261022 ... 5.77802903 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1364
테스트 세트 예측:
[5.92082102 6.         5.63239793 ... 5.77769791 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1365
테스트 세트 예측:
[5.92085159 6.         5.63252119 ... 5.77821789 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1366
테스트 세트 예측:
[5.92088213 6.         5.63230923 ... 5.77831207 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1367
테스트 세트 예측:
[5.92129837 6.         5.63209749 ... 5.77840614 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1368
테스트 세트 예측:
[5.92094346 6.         5.63255466 ... 5.778076   6.         5.        ]
테스트 세트 R^2: 0.29
n =  1369
테스트 세트 예측:
[5.92097388 6.         5.63267738 ... 5.77774649 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1370
테스트 세트 예측:
[5.9217736  6.         5.63279973 ... 5.77741727 6.         5.        ]
테스트 세트 R^2

테스트 세트 R^2: 0.29
n =  1437
테스트 세트 예측:
[5.92621006 6.         5.63878776 ... 5.77940939 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1438
테스트 세트 예측:
[5.92696043 6.         5.63890159 ... 5.77909656 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1439
테스트 세트 예측:
[5.92698685 6.         5.63932996 ... 5.77918518 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1440
테스트 세트 예측:
[5.92665181 6.         5.63944344 ... 5.77927374 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1441
테스트 세트 예측:
[5.92631714 6.         5.63955682 ... 5.77936221 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1442
테스트 세트 예측:
[5.92670475 6.         5.63967003 ... 5.77985103 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1443
테스트 세트 예측:
[5.92709153 6.         5.63978313 ... 5.77993909 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1444
테스트 세트 예측:
[5.92747802 6.         5.63989616 ... 5.78002709 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1445
테스트 세트 예측:
[5.92750411 6.         5.64000901 ... 5.78011501 6.         5.        ]
테스트 세트 R^2

테스트 세트 R^2: 0.29
n =  1512
테스트 세트 예측:
[5.93162042 6.         5.64272974 ... 5.78384656 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1513
테스트 세트 예측:
[5.93164359 6.         5.64253895 ... 5.78392811 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1514
테스트 세트 예측:
[5.93166673 6.         5.64234826 ... 5.78438662 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1515
테스트 세트 예측:
[5.93168984 6.         5.6421577  ... 5.78409114 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1516
테스트 세트 예측:
[5.93137513 6.         5.64196749 ... 5.78417242 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1517
테스트 세트 예측:
[5.93173557 6.         5.64207339 ... 5.78387734 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1518
테스트 세트 예측:
[5.93142177 6.         5.64247428 ... 5.78433469 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1519
테스트 세트 예측:
[5.93110818 6.         5.64257978 ... 5.78479169 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1520
테스트 세트 예측:
[5.93113134 6.         5.64268522 ... 5.78487254 6.         5.        ]
테스트 세트 R^2

테스트 세트 R^2: 0.29
n =  1587
테스트 세트 예측:
[5.93229677 6.         5.64805073 ... 5.78355991 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1588
테스트 세트 예측:
[5.9323182  6.         5.64787194 ... 5.78363741 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1589
테스트 세트 예측:
[5.93265552 6.         5.64824487 ... 5.78407263 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1590
테스트 세트 예측:
[5.93299257 6.         5.64834158 ... 5.78414984 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1591
테스트 세트 예측:
[5.9330137  6.         5.64843821 ... 5.78386957 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1592
테스트 세트 예측:
[5.93303482 6.         5.64853469 ... 5.78394669 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1593
테스트 세트 예측:
[5.93337096 6.         5.64863111 ... 5.78402374 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1594
테스트 세트 예측:
[5.93339191 6.         5.64845333 ... 5.78481372 6.         5.        ]
테스트 세트 R^2: 0.29
n =  1595
테스트 세트 예측:
[5.93309876 6.         5.64827576 ... 5.78417797 6.         5.        ]
테스트 세트 R^2

### 1-4 / 5-10

In [7]:
# 훈련 데이터와 테스트 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(total_data, target_5, random_state=0)
# 훈련 데이터 크기 확인
print(x_train.shape)
print(y_train.shape)
# 테스트 데이터 크기 확인
print(x_test.shape)
print(y_test.shape)

(4872, 12)
(4872,)
(1625, 12)
(1625,)


In [15]:
l=[]# 반환된 정확도를 모아둘 배열
for i in range(1,100):#가장 가까운 1개 부터 시작해서 자신을 제외한 모든 것에 대해 kNN 시행(전부 다 하면 진짜 오래 걸립니다ㅠㅜ)
    #최대 k = 1625
    l.append(knr_by_n(i,x_train,y_train,x_test,y_test))
best=float(l[0])
for i in l:
    if i>best:
        best=i
print(best)
print("k가 " + str(l.index(best)+1) + "일때 정확도 " + str(best) + "가 최대 정확도")

n =  1
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.27
n =  2
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.10
n =  3
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.01
n =  4
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.04
n =  5
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.05
n =  6
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.07
n =  7
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.07
n =  8
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.08
n =  9
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.08
n =  10
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.09
n =  11
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.09
n =  12
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  13
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  14
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  15
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.11
n =  16
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  17
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트

n =  88
테스트 세트 예측:
[0.97751017 1.         0.97782743 ... 0.95350756 1.         1.        ]
테스트 세트 R^2: 0.10
n =  89
테스트 세트 예측:
[0.97769339 1.         0.97800239 ... 0.95385764 1.         1.        ]
테스트 세트 R^2: 0.10
n =  90
테스트 세트 예측:
[0.9778731  1.         0.97817386 ... 0.95420245 1.         1.        ]
테스트 세트 R^2: 0.10
n =  91
테스트 세트 예측:
[0.97804991 1.         0.9783402  ... 0.95454208 1.         1.        ]
테스트 세트 R^2: 0.10
n =  92
테스트 세트 예측:
[0.97822363 1.         0.97850175 ... 0.95487628 1.         1.        ]
테스트 세트 R^2: 0.10
n =  93
테스트 세트 예측:
[0.97839302 1.         0.9786609  ... 0.95520363 1.         1.        ]
테스트 세트 R^2: 0.10
n =  94
테스트 세트 예측:
[0.97855953 1.         0.97881736 ... 0.95552274 1.         1.        ]
테스트 세트 R^2: 0.10
n =  95
테스트 세트 예측:
[0.97872301 1.         0.97897155 ... 0.94878647 1.         1.        ]
테스트 세트 R^2: 0.10
n =  96
테스트 세트 예측:
[0.97888401 1.         0.97912332 ... 0.94914468 1.         1.        ]
테스트 세트 R^2: 0.10
n =  97
테스트 세트 예측:
[0.979041

In [16]:
print(l)

[-0.26602564102564097, -0.10028962811107656, -0.012330372946150803, 0.03550705480050875, 0.05364326504762629, 0.06889193798752813, 0.07453687460385783, 0.07796710772806759, 0.07869352185587342, 0.09391408587693428, 0.09111563411416768, 0.09930460279079756, 0.09979477705215645, 0.10261647276190067, 0.10545460141769392, 0.10472916042669178, 0.10500956960360286, 0.10706814692013733, 0.10759037817603188, 0.10636539254008381, 0.10881288734968131, 0.10949013551432274, 0.1084982571448706, 0.10795229995852673, 0.10733914492244223, 0.10628985581079986, 0.10699811648563207, 0.10686923102745616, 0.10656180530053838, 0.10852526120125638, 0.107190153424087, 0.10697154450915314, 0.10868386401076036, 0.10742871713109846, 0.106670478338866, 0.10751506722359772, 0.10781218962759387, 0.10684693125629041, 0.10695623042166269, 0.1052919128301959, 0.10449060182488312, 0.10349893172626157, 0.10279163080888498, 0.103108859389197, 0.10225433498968806, 0.10272535111589587, 0.1034434819258202, 0.105372583887090

### 1-5 / 6-10

In [9]:
# 훈련 데이터와 테스트 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(total_data, target_6, random_state=0)
# 훈련 데이터 크기 확인
print(x_train.shape)
print(y_train.shape)
# 테스트 데이터 크기 확인
print(x_test.shape)
print(y_test.shape)

(4872, 12)
(4872,)
(1625, 12)
(1625,)


In [10]:

for i in range(1,1625):#가장 가까운 1개 부터 시작해서 자신을 제외한 모든 것에 대해 kNN 시행(전부 다 하면 진짜 오래 걸립니다ㅠㅜ)
    #최대 k = 1625
    knr_by_n(i,x_train,y_train,x_test,y_test)

n =  1
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.27
n =  2
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.10
n =  3
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.01
n =  4
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.04
n =  5
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.05
n =  6
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.07
n =  7
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.07
n =  8
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.08
n =  9
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.08
n =  10
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.09
n =  11
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.09
n =  12
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  13
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  14
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  15
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.11
n =  16
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  17
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트

테스트 세트 R^2: 0.10
n =  89
테스트 세트 예측:
[0.97769339 1.         0.97800239 ... 0.95385764 1.         1.        ]
테스트 세트 R^2: 0.10
n =  90
테스트 세트 예측:
[0.9778731  1.         0.97817386 ... 0.95420245 1.         1.        ]
테스트 세트 R^2: 0.10
n =  91
테스트 세트 예측:
[0.97804991 1.         0.9783402  ... 0.95454208 1.         1.        ]
테스트 세트 R^2: 0.10
n =  92
테스트 세트 예측:
[0.97822363 1.         0.97850175 ... 0.95487628 1.         1.        ]
테스트 세트 R^2: 0.10
n =  93
테스트 세트 예측:
[0.97839302 1.         0.9786609  ... 0.95520363 1.         1.        ]
테스트 세트 R^2: 0.10
n =  94
테스트 세트 예측:
[0.97855953 1.         0.97881736 ... 0.95552274 1.         1.        ]
테스트 세트 R^2: 0.10
n =  95
테스트 세트 예측:
[0.97872301 1.         0.97897155 ... 0.94878647 1.         1.        ]
테스트 세트 R^2: 0.10
n =  96
테스트 세트 예측:
[0.97888401 1.         0.97912332 ... 0.94914468 1.         1.        ]
테스트 세트 R^2: 0.10
n =  97
테스트 세트 예측:
[0.97904131 1.         0.97927269 ... 0.94949778 1.         1.        ]
테스트 세트 R^2: 0.10
n =  98
테스트

테스트 세트 R^2: 0.10
n =  165
테스트 세트 예측:
[0.98092957 1.         0.97148861 ... 0.96001405 1.         1.        ]
테스트 세트 R^2: 0.10
n =  166
테스트 세트 예측:
[0.981009   1.         0.97159954 ... 0.96017942 1.         1.        ]
테스트 세트 R^2: 0.10
n =  167
테스트 세트 예측:
[0.98108759 1.         0.97170961 ... 0.96034272 1.         1.        ]
테스트 세트 R^2: 0.10
n =  168
테스트 세트 예측:
[0.98116539 1.         0.97181851 ... 0.96050418 1.         1.        ]
테스트 세트 R^2: 0.10
n =  169
테스트 세트 예측:
[0.98124249 1.         0.97192645 ... 0.96066393 1.         1.        ]
테스트 세트 R^2: 0.10
n =  170
테스트 세트 예측:
[0.98131878 1.         0.97203248 ... 0.9608224  1.         1.        ]
테스트 세트 R^2: 0.10
n =  171
테스트 세트 예측:
[0.97735733 1.         0.97213768 ... 0.96097891 1.         1.        ]
테스트 세트 R^2: 0.10
n =  172
테스트 세트 예측:
[0.97342774 1.         0.97224203 ... 0.96113412 1.         1.        ]
테스트 세트 R^2: 0.10
n =  173
테스트 세트 예측:
[0.97353412 1.         0.97234543 ... 0.9612879  1.         1.        ]
테스트 세트 R^2: 0.10
n 

테스트 세트 예측:
[0.96102311 1.         0.974716   ... 0.96904357 1.         1.        ]
테스트 세트 R^2: 0.10
n =  241
테스트 세트 예측:
[0.96113135 1.         0.9747808  ... 0.96912955 1.         1.        ]
테스트 세트 R^2: 0.10
n =  242
테스트 세트 예측:
[0.96123888 1.         0.97484516 ... 0.96921484 1.         1.        ]
테스트 세트 R^2: 0.10
n =  243
테스트 세트 예측:
[0.96134577 1.         0.97490912 ... 0.9692995  1.         1.        ]
테스트 세트 R^2: 0.10
n =  244
테스트 세트 예측:
[0.96145155 1.         0.97497241 ... 0.96938361 1.         1.        ]
테스트 세트 R^2: 0.10
n =  245
테스트 세트 예측:
[0.9615567  1.         0.97503519 ... 0.96946722 1.         1.        ]
테스트 세트 R^2: 0.10
n =  246
테스트 세트 예측:
[0.96166114 1.         0.97509748 ... 0.96955037 1.         1.        ]
테스트 세트 R^2: 0.10
n =  247
테스트 세트 예측:
[0.96176472 1.         0.97515936 ... 0.96963308 1.         1.        ]
테스트 세트 R^2: 0.10
n =  248
테스트 세트 예측:
[0.96186745 1.         0.97522093 ... 0.96971529 1.         1.        ]
테스트 세트 R^2: 0.10
n =  249
테스트 세트 예측:
[0.96196

테스트 세트 R^2: 0.09
n =  316
테스트 세트 예측:
[0.96740002 1.         0.97663805 ... 0.96311369 1.         1.        ]
테스트 세트 R^2: 0.09
n =  317
테스트 세트 예측:
[0.96746595 1.         0.9766823  ... 0.9631897  1.         1.        ]
테스트 세트 R^2: 0.09
n =  318
테스트 세트 예측:
[0.96753154 1.         0.97672612 ... 0.96326538 1.         1.        ]
테스트 세트 R^2: 0.09
n =  319
테스트 세트 예측:
[0.96759682 1.         0.97676977 ... 0.96334061 1.         1.        ]
테스트 세트 R^2: 0.09
n =  320
테스트 세트 예측:
[0.96766146 1.         0.97681321 ... 0.96341527 1.         1.        ]
테스트 세트 R^2: 0.09
n =  321
테스트 세트 예측:
[0.96772583 1.         0.97685633 ... 0.96348955 1.         1.        ]
테스트 세트 R^2: 0.09
n =  322
테스트 세트 예측:
[0.96778989 1.         0.9768993  ... 0.96356348 1.         1.        ]
테스트 세트 R^2: 0.09
n =  323
테스트 세트 예측:
[0.96785361 1.         0.97694208 ... 0.96363691 1.         1.        ]
테스트 세트 R^2: 0.09
n =  324
테스트 세트 예측:
[0.96791703 1.         0.97698462 ... 0.9637099  1.         1.        ]
테스트 세트 R^2: 0.09
n 

...

테스트 세트 R^2: 0.08
n =  1297
테스트 세트 예측:
[0.96865479 1.         0.97435356 ... 0.96964459 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1298
테스트 세트 예측:
[0.96866759 1.         0.97436268 ... 0.96965827 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1299
테스트 세트 예측:
[0.96868039 1.         0.97437179 ... 0.96967194 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1300
테스트 세트 예측:
[0.968285   1.         0.9743809  ... 0.96968559 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1301
테스트 세트 예측:
[0.96829794 1.         0.97438999 ... 0.96969923 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1302
테스트 세트 예측:
[0.96831084 1.         0.97439907 ... 0.96971284 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1303
테스트 세트 예측:
[0.96832372 1.         0.97440815 ... 0.96972644 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1304
테스트 세트 예측:
[0.96833659 1.         0.97441721 ... 0.96974002 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1305
테스트 세트 예측:
[0.96834942 1.         0.97442627 ... 0.96975359 1.         1.        ]
테스트 세트 R^2

테스트 세트 R^2: 0.08
n =  1372
테스트 세트 예측:
[0.96917549 1.         0.97466424 ... 0.97062204 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1373
테스트 세트 예측:
[0.96918731 1.         0.97467266 ... 0.97063443 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1374
테스트 세트 예측:
[0.96919907 1.         0.97468107 ... 0.97064681 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1375
테스트 세트 예측:
[0.96921081 1.         0.97468947 ... 0.97065919 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1376
테스트 세트 예측:
[0.96922251 1.         0.97469787 ... 0.97067154 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1377
테스트 세트 예측:
[0.9692342  1.         0.97470626 ... 0.97068387 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1378
테스트 세트 예측:
[0.96924588 1.         0.97471462 ... 0.97027577 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1379
테스트 세트 예측:
[0.96925755 1.         0.97472298 ... 0.97028825 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1380
테스트 세트 예측:
[0.96926921 1.         0.97473132 ... 0.97030072 1.         1.        ]
테스트 세트 R^2

테스트 세트 R^2: 0.08
n =  1447
테스트 세트 예측:
[0.96965215 1.         0.9746301  ... 0.97110421 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1448
테스트 세트 예측:
[0.96966305 1.         0.97463803 ... 0.97111574 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1449
테스트 세트 예측:
[0.96967394 1.         0.97464596 ... 0.97112725 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1450
테스트 세트 예측:
[0.96968481 1.         0.97465387 ... 0.97113876 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1451
테스트 세트 예측:
[0.96969566 1.         0.97466177 ... 0.97115026 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1452
테스트 세트 예측:
[0.96970651 1.         0.97466966 ... 0.97116173 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1453
테스트 세트 예측:
[0.96971734 1.         0.97467754 ... 0.97117319 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1454
테스트 세트 예측:
[0.96972815 1.         0.97468542 ... 0.97118461 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1455
테스트 세트 예측:
[0.96973896 1.         0.97469328 ... 0.97119603 1.         1.        ]
테스트 세트 R^2

테스트 세트 R^2: 0.07
n =  1522
테스트 세트 예측:
[0.97043317 1.         0.97429272 ... 0.97155    1.         1.        ]
테스트 세트 R^2: 0.07
n =  1523
테스트 세트 예측:
[0.97044309 1.         0.97430027 ... 0.97156066 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1524
테스트 세트 예측:
[0.970453   1.         0.97430782 ... 0.97157129 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1525
테스트 세트 예측:
[0.97046289 1.         0.97431535 ... 0.9715819  1.         1.        ]
테스트 세트 R^2: 0.07
n =  1526
테스트 세트 예측:
[0.97047278 1.         0.97432288 ... 0.97159251 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1527
테스트 세트 예측:
[0.97048265 1.         0.9743304  ... 0.9716031  1.         1.        ]
테스트 세트 R^2: 0.07
n =  1528
테스트 세트 예측:
[0.97049251 1.         0.97433792 ... 0.97161368 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1529
테스트 세트 예측:
[0.97050235 1.         0.97434542 ... 0.97162426 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1530
테스트 세트 예측:
[0.97051218 1.         0.97435291 ... 0.97163482 1.         1.        ]
테스트 세트 R^2

테스트 세트 R^2: 0.07
n =  1597
테스트 세트 예측:
[0.97081835 1.         0.9745525  ... 0.9716068  1.         1.        ]
테스트 세트 R^2: 0.07
n =  1598
테스트 세트 예측:
[0.97082749 1.         0.97455944 ... 0.97161689 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1599
테스트 세트 예측:
[0.97083662 1.         0.97456639 ... 0.97162696 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1600
테스트 세트 예측:
[0.97084573 1.         0.97457333 ... 0.97163703 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1601
테스트 세트 예측:
[0.97085484 1.         0.97458026 ... 0.97164709 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1602
테스트 세트 예측:
[0.97086394 1.         0.97458718 ... 0.97165713 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1603
테스트 세트 예측:
[0.97087304 1.         0.9745941  ... 0.97166715 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1604
테스트 세트 예측:
[0.97088212 1.         0.97460101 ... 0.97167717 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1605
테스트 세트 예측:
[0.9708912  1.         0.9746079  ... 0.97168717 1.         1.        ]
테스트 세트 R^2

### 1-6 / 7-10

In [11]:
# 훈련 데이터와 테스트 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(total_data, target_7, random_state=0)
# 훈련 데이터 크기 확인
print(x_train.shape)
print(y_train.shape)
# 테스트 데이터 크기 확인
print(x_test.shape)
print(y_test.shape)

(4872, 12)
(4872,)
(1625, 12)
(1625,)


In [12]:
l1=[]# 반환된 정확도를 모아둘 배열
for i in range(1,1625):
    #최대 k = 1625
    l1.append(knr_by_n(i,x_train,y_train,x_test,y_test))
best=0.0
for i in l1:
    if i>best:
        best=i
print(best)
print("k가 " + str(l1.index(best)+1) + "일때 정확도 " + str(best) + "가 최대 정확도")

n =  1
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.27
n =  2
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.10
n =  3
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.01
n =  4
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.04
n =  5
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.05
n =  6
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.07
n =  7
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.07
n =  8
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.08
n =  9
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.08
n =  10
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.09
n =  11
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.09
n =  12
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  13
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  14
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  15
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.11
n =  16
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  17
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트

테스트 세트 R^2: 0.10
n =  88
테스트 세트 예측:
[0.97751017 1.         0.97782743 ... 0.95350756 1.         1.        ]
테스트 세트 R^2: 0.10
n =  89
테스트 세트 예측:
[0.97769339 1.         0.97800239 ... 0.95385764 1.         1.        ]
테스트 세트 R^2: 0.10
n =  90
테스트 세트 예측:
[0.9778731  1.         0.97817386 ... 0.95420245 1.         1.        ]
테스트 세트 R^2: 0.10
n =  91
테스트 세트 예측:
[0.97804991 1.         0.9783402  ... 0.95454208 1.         1.        ]
테스트 세트 R^2: 0.10
n =  92
테스트 세트 예측:
[0.97822363 1.         0.97850175 ... 0.95487628 1.         1.        ]
테스트 세트 R^2: 0.10
n =  93
테스트 세트 예측:
[0.97839302 1.         0.9786609  ... 0.95520363 1.         1.        ]
테스트 세트 R^2: 0.10
n =  94
테스트 세트 예측:
[0.97855953 1.         0.97881736 ... 0.95552274 1.         1.        ]
테스트 세트 R^2: 0.10
n =  95
테스트 세트 예측:
[0.97872301 1.         0.97897155 ... 0.94878647 1.         1.        ]
테스트 세트 R^2: 0.10
n =  96
테스트 세트 예측:
[0.97888401 1.         0.97912332 ... 0.94914468 1.         1.        ]
테스트 세트 R^2: 0.10
n =  97
테스트

테스트 세트 예측:
[0.98084928 1.         0.97137599 ... 0.95984729 1.         1.        ]
테스트 세트 R^2: 0.10
n =  165
테스트 세트 예측:
[0.98092957 1.         0.97148861 ... 0.96001405 1.         1.        ]
테스트 세트 R^2: 0.10
n =  166
테스트 세트 예측:
[0.981009   1.         0.97159954 ... 0.96017942 1.         1.        ]
테스트 세트 R^2: 0.10
n =  167
테스트 세트 예측:
[0.98108759 1.         0.97170961 ... 0.96034272 1.         1.        ]
테스트 세트 R^2: 0.10
n =  168
테스트 세트 예측:
[0.98116539 1.         0.97181851 ... 0.96050418 1.         1.        ]
테스트 세트 R^2: 0.10
n =  169
테스트 세트 예측:
[0.98124249 1.         0.97192645 ... 0.96066393 1.         1.        ]
테스트 세트 R^2: 0.10
n =  170
테스트 세트 예측:
[0.98131878 1.         0.97203248 ... 0.9608224  1.         1.        ]
테스트 세트 R^2: 0.10
n =  171
테스트 세트 예측:
[0.97735733 1.         0.97213768 ... 0.96097891 1.         1.        ]
테스트 세트 R^2: 0.10
n =  172
테스트 세트 예측:
[0.97342774 1.         0.97224203 ... 0.96113412 1.         1.        ]
테스트 세트 R^2: 0.10
n =  173
테스트 세트 예측:
[0.97353

테스트 세트 R^2: 0.10
n =  241
테스트 세트 예측:
[0.96113135 1.         0.9747808  ... 0.96912955 1.         1.        ]
테스트 세트 R^2: 0.10
n =  242
테스트 세트 예측:
[0.96123888 1.         0.97484516 ... 0.96921484 1.         1.        ]
테스트 세트 R^2: 0.10
n =  243
테스트 세트 예측:
[0.96134577 1.         0.97490912 ... 0.9692995  1.         1.        ]
테스트 세트 R^2: 0.10
n =  244
테스트 세트 예측:
[0.96145155 1.         0.97497241 ... 0.96938361 1.         1.        ]
테스트 세트 R^2: 0.10
n =  245
테스트 세트 예측:
[0.9615567  1.         0.97503519 ... 0.96946722 1.         1.        ]
테스트 세트 R^2: 0.10
n =  246
테스트 세트 예측:
[0.96166114 1.         0.97509748 ... 0.96955037 1.         1.        ]
테스트 세트 R^2: 0.10
n =  247
테스트 세트 예측:
[0.96176472 1.         0.97515936 ... 0.96963308 1.         1.        ]
테스트 세트 R^2: 0.10
n =  248
테스트 세트 예측:
[0.96186745 1.         0.97522093 ... 0.96971529 1.         1.        ]
테스트 세트 R^2: 0.10
n =  249
테스트 세트 예측:
[0.96196918 1.         0.9752822  ... 0.96979698 1.         1.        ]
테스트 세트 R^2: 0.10
n 

테스트 세트 R^2: 0.09
n =  317
테스트 세트 예측:
[0.96746595 1.         0.9766823  ... 0.9631897  1.         1.        ]
테스트 세트 R^2: 0.09
n =  318
테스트 세트 예측:
[0.96753154 1.         0.97672612 ... 0.96326538 1.         1.        ]
테스트 세트 R^2: 0.09
n =  319
테스트 세트 예측:
[0.96759682 1.         0.97676977 ... 0.96334061 1.         1.        ]
테스트 세트 R^2: 0.09
n =  320
테스트 세트 예측:
[0.96766146 1.         0.97681321 ... 0.96341527 1.         1.        ]
테스트 세트 R^2: 0.09
n =  321
테스트 세트 예측:
[0.96772583 1.         0.97685633 ... 0.96348955 1.         1.        ]
테스트 세트 R^2: 0.09
n =  322
테스트 세트 예측:
[0.96778989 1.         0.9768993  ... 0.96356348 1.         1.        ]
테스트 세트 R^2: 0.09
n =  323
테스트 세트 예측:
[0.96785361 1.         0.97694208 ... 0.96363691 1.         1.        ]
테스트 세트 R^2: 0.09
n =  324
테스트 세트 예측:
[0.96791703 1.         0.97698462 ... 0.9637099  1.         1.        ]
테스트 세트 R^2: 0.09
n =  325
테스트 세트 예측:
[0.96798008 1.         0.97702701 ... 0.96378219 1.         1.        ]
테스트 세트 R^2: 0.09
n 

...

테스트 세트 R^2: 0.08
n =  1373
테스트 세트 예측:
[0.96918731 1.         0.97467266 ... 0.97063443 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1374
테스트 세트 예측:
[0.96919907 1.         0.97468107 ... 0.97064681 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1375
테스트 세트 예측:
[0.96921081 1.         0.97468947 ... 0.97065919 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1376
테스트 세트 예측:
[0.96922251 1.         0.97469787 ... 0.97067154 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1377
테스트 세트 예측:
[0.9692342  1.         0.97470626 ... 0.97068387 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1378
테스트 세트 예측:
[0.96924588 1.         0.97471462 ... 0.97027577 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1379
테스트 세트 예측:
[0.96925755 1.         0.97472298 ... 0.97028825 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1380
테스트 세트 예측:
[0.96926921 1.         0.97473132 ... 0.97030072 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1381
테스트 세트 예측:
[0.96928084 1.         0.97473966 ... 0.97031318 1.         1.        ]
테스트 세트 R^2

테스트 세트 R^2: 0.08
n =  1448
테스트 세트 예측:
[0.96966305 1.         0.97463803 ... 0.97111574 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1449
테스트 세트 예측:
[0.96967394 1.         0.97464596 ... 0.97112725 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1450
테스트 세트 예측:
[0.96968481 1.         0.97465387 ... 0.97113876 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1451
테스트 세트 예측:
[0.96969566 1.         0.97466177 ... 0.97115026 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1452
테스트 세트 예측:
[0.96970651 1.         0.97466966 ... 0.97116173 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1453
테스트 세트 예측:
[0.96971734 1.         0.97467754 ... 0.97117319 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1454
테스트 세트 예측:
[0.96972815 1.         0.97468542 ... 0.97118461 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1455
테스트 세트 예측:
[0.96973896 1.         0.97469328 ... 0.97119603 1.         1.        ]
테스트 세트 R^2: 0.08
n =  1456
테스트 세트 예측:
[0.96974976 1.         0.97470114 ... 0.97120744 1.         1.        ]
테스트 세트 R^2

테스트 세트 R^2: 0.07
n =  1523
테스트 세트 예측:
[0.97044309 1.         0.97430027 ... 0.97156066 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1524
테스트 세트 예측:
[0.970453   1.         0.97430782 ... 0.97157129 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1525
테스트 세트 예측:
[0.97046289 1.         0.97431535 ... 0.9715819  1.         1.        ]
테스트 세트 R^2: 0.07
n =  1526
테스트 세트 예측:
[0.97047278 1.         0.97432288 ... 0.97159251 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1527
테스트 세트 예측:
[0.97048265 1.         0.9743304  ... 0.9716031  1.         1.        ]
테스트 세트 R^2: 0.07
n =  1528
테스트 세트 예측:
[0.97049251 1.         0.97433792 ... 0.97161368 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1529
테스트 세트 예측:
[0.97050235 1.         0.97434542 ... 0.97162426 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1530
테스트 세트 예측:
[0.97051218 1.         0.97435291 ... 0.97163482 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1531
테스트 세트 예측:
[0.970522   1.         0.97436039 ... 0.97164538 1.         1.        ]
테스트 세트 R^2

테스트 세트 R^2: 0.07
n =  1598
테스트 세트 예측:
[0.97082749 1.         0.97455944 ... 0.97161689 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1599
테스트 세트 예측:
[0.97083662 1.         0.97456639 ... 0.97162696 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1600
테스트 세트 예측:
[0.97084573 1.         0.97457333 ... 0.97163703 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1601
테스트 세트 예측:
[0.97085484 1.         0.97458026 ... 0.97164709 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1602
테스트 세트 예측:
[0.97086394 1.         0.97458718 ... 0.97165713 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1603
테스트 세트 예측:
[0.97087304 1.         0.9745941  ... 0.97166715 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1604
테스트 세트 예측:
[0.97088212 1.         0.97460101 ... 0.97167717 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1605
테스트 세트 예측:
[0.9708912  1.         0.9746079  ... 0.97168717 1.         1.        ]
테스트 세트 R^2: 0.07
n =  1606
테스트 세트 예측:
[0.97090027 1.         0.97461479 ... 0.97169716 1.         1.        ]
테스트 세트 R^2

TypeError: '>' not supported between instances of 'NoneType' and 'float'

# 다중분류: 0.35

# 이진분류: 0.11